In [1]:
# 외부의 텍스트 파일을 로드하는 방법
text = open('../data/qa.txt', 'r', encoding='utf-8').read()
# 'r' : 읽기 전용
text        # str

'[\n    ("환불은 어떻게 하나요?", "주문 상세 페이지에서 \'환불 신청\' 버튼을 눌러 접수하실 수 있습니다."),\n    ("배송 기간은 얼마나 걸리나요?", "일반 배송은 2~3일, 도서산간 지역은 최대 5일까지 소요됩니다."),\n    ("해외 배송도 가능한가요?", "현재 해외 배송은 지원하지 않습니다."),\n    ("회원 탈퇴는 어디에서 하나요?", "설정 > 계정 관리 > 회원 탈퇴 메뉴에서 진행하실 수 있습니다."),\n    ("비밀번호를 잊어버렸어요", "로그인 화면의 \'비밀번호 재설정\' 링크를 통해 재설정 가능합니다."),\n    ("주문 취소는 어떻게 하죠?", "상품이 배송 준비 전 상태라면 주문 상세 페이지에서 취소가 가능합니다."),\n    ("영수증 발급이 가능한가요?", "마이페이지 > 주문 내역에서 영수증 출력이 가능합니다."),\n    ("교환/반품은 가능한가요?", "수령일로부터 7일 이내, 미사용/미훼손 제품에 한해 가능합니다."),\n]'

In [2]:
# eval(): 일반 문자를 python의 구조로 변경하는 함수
# "" - 문자, [] - 리스트, () - 튜플로 인식
QnA_list = eval(text)
QnA_list    # list

[('환불은 어떻게 하나요?', "주문 상세 페이지에서 '환불 신청' 버튼을 눌러 접수하실 수 있습니다."),
 ('배송 기간은 얼마나 걸리나요?', '일반 배송은 2~3일, 도서산간 지역은 최대 5일까지 소요됩니다.'),
 ('해외 배송도 가능한가요?', '현재 해외 배송은 지원하지 않습니다.'),
 ('회원 탈퇴는 어디에서 하나요?', '설정 > 계정 관리 > 회원 탈퇴 메뉴에서 진행하실 수 있습니다.'),
 ('비밀번호를 잊어버렸어요', "로그인 화면의 '비밀번호 재설정' 링크를 통해 재설정 가능합니다."),
 ('주문 취소는 어떻게 하죠?', '상품이 배송 준비 전 상태라면 주문 상세 페이지에서 취소가 가능합니다.'),
 ('영수증 발급이 가능한가요?', '마이페이지 > 주문 내역에서 영수증 출력이 가능합니다.'),
 ('교환/반품은 가능한가요?', '수령일로부터 7일 이내, 미사용/미훼손 제품에 한해 가능합니다.')]

- QnA_list에서 질문들을 따로 추출하여 형태소 분석을 통해 단어 추출
- 벡터화
- 코사인 유사도
    - 문장끼리 어느 정도의 같은 의미를 가지는가?

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from konlpy.tag import Komoran

In [4]:
# QnA_list (질문, 답) 에서 질문을 추출해 목록 생성
# 패턴 인지 -> list에서 각각의 원소들을 추출 -> 원소에서 첫 번째 문자를 추출
# 질문을 담을 새로운 리스트 생성
questions = []

# list에서 각각의 원소들을 추출
# 방법 1
for data in QnA_list:
    print(data[0])

환불은 어떻게 하나요?
배송 기간은 얼마나 걸리나요?
해외 배송도 가능한가요?
회원 탈퇴는 어디에서 하나요?
비밀번호를 잊어버렸어요
주문 취소는 어떻게 하죠?
영수증 발급이 가능한가요?
교환/반품은 가능한가요?


In [5]:
# 방법 2
for q, a in QnA_list:
    print(q)

환불은 어떻게 하나요?
배송 기간은 얼마나 걸리나요?
해외 배송도 가능한가요?
회원 탈퇴는 어디에서 하나요?
비밀번호를 잊어버렸어요
주문 취소는 어떻게 하죠?
영수증 발급이 가능한가요?
교환/반품은 가능한가요?


In [6]:
# 방법 3
questions = [ q for q, a in QnA_list ]
questions

['환불은 어떻게 하나요?',
 '배송 기간은 얼마나 걸리나요?',
 '해외 배송도 가능한가요?',
 '회원 탈퇴는 어디에서 하나요?',
 '비밀번호를 잊어버렸어요',
 '주문 취소는 어떻게 하죠?',
 '영수증 발급이 가능한가요?',
 '교환/반품은 가능한가요?']

In [7]:
# 토큰화
komoran = Komoran()

def tokenize(text):
    return komoran.morphs(text)

# 벡터화
vectorizer = TfidfVectorizer(
    tokenizer= tokenize,
    ngram_range= (1, 2),
    lowercase= False
)

In [8]:
X = vectorizer.fit_transform(questions)

c:\Users\student\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\feature_extraction\text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [9]:
X.toarray().shape

(8, 76)

In [10]:
# 새로운 질문
query = "배송지 변경을 하려면 어떻게 하면 될까요?"

# 질문을 벡터화
query_vec = vectorizer.transform( [query] )

In [11]:
# 코사인 거리 유사도 함수
# ravel() : array에서 사용하는 함수로, 다차원 배열을 1차원 배열로 변경하는 함수
sims = cosine_similarity(query_vec, X).ravel()
sims

array([0.60002384, 0.02818594, 0.11787635, 0.10117384, 0.        ,
       0.50670911, 0.1159249 , 0.10726304])

In [ ]:
# argsort() : 배열의 요소를 오름차순으로 정렬했을 때, 각 요소가 위치하게 되는 인덱스 배열을 반환하는 함수
# 값 자체가 아니라, 값이 정렬되었을 때의 원래 위치(인덱스)를 알고 싶을 때 사용
# 순위를 기반으로 데이터를 선택해야 할 때 매우 유용
# 정렬 순서는 인덱스의 의미
# ::-1 = 처음부터, 끝까지, 반대 순서로
rank = sims.argsort()[::-1]
rank

array([0, 5, 2, 6, 7, 3, 1, 4])

In [13]:
print('질문: ', query)
for i in rank[:2]:
    print(f"Index: {i}, 유사도: {sims[i] :.3f}, 유사 질문: {questions[i]}, 답변: {QnA_list[i][1]}")

질문:  배송지 변경을 하려면 어떻게 하면 될까요?
Index: 0, 유사도: 0.600, 유사 질문: 환불은 어떻게 하나요?, 답변: 주문 상세 페이지에서 '환불 신청' 버튼을 눌러 접수하실 수 있습니다.
Index: 5, 유사도: 0.507, 유사 질문: 주문 취소는 어떻게 하죠?, 답변: 상품이 배송 준비 전 상태라면 주문 상세 페이지에서 취소가 가능합니다.
